In [1]:
import sys
import os
from IPython.display import clear_output
import cv2
import cvzone
import math
from sort import *
from tqdm import tqdm
from ultralytics import YOLO
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


# Fonction de détection et tracking

In [19]:
def detect_and_track(FILE_PATH_IMG, FILE_PATH_RESULTS,file_name = "/results.txt" , sort = False, show = False, className="person", max_age=20, min_hits=1, iou_threshold=0.3, conf_yolo=0.5):

    model = YOLO("yolov8l.pt")
    classNames = model.names
    tracker = Sort(max_age=max_age, min_hits=min_hits,iou_threshold=iou_threshold)
    
    folder_files_img = os.listdir(FILE_PATH_IMG)
    if sort:
        folder_files_img = sorted(folder_files_img, key=lambda x: int(x[5:-4]))

    if not os.path.exists(FILE_PATH_RESULTS):
        os.makedirs(FILE_PATH_RESULTS)
    f = open(FILE_PATH_RESULTS+file_name, "w")

    for frame, picture in enumerate(tqdm(folder_files_img), 1):
        img = cv2.imread(f"{FILE_PATH_IMG}/{picture}")
        result = model(img, verbose=False)
        detections = np.empty((0,5))
        
        # conf_list = []
        
        for r in result:
            boxes = r.boxes
            for box in boxes:
                #Score de confiance
                conf = math.ceil(box.conf[0]*100)/100
                
                #For bounding box
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                # bbox = x1, y1, x2, y2
                # w, h = x2-x1, y2-y1
                
                #For confidence box
                cls = int(box.cls[0])
                curr_class = classNames[cls]
                
                # conf_list.append(conf)
                
                if curr_class == className and conf > conf_yolo:
                    if show:
                        cvzone.putTextRect(img, f"{model.names[cls]} {conf}", (x2, y2), scale=1, thickness=1, colorR=(0,0,255))
                    currentArray = np.array([[x1, y1, x2, y2, conf]])
                    detections = np.vstack((detections, currentArray))
        
        results_tracker = tracker.update(detections)
        
        for i, res in enumerate(results_tracker):
            x1, y1, x2, y2, id = res
            x1, y1, x2, y2, id = int(x1), int(y1), int(x2), int(y2), int(id)
            w, h = x2-x1, y2-y1
            
            if show:
                cvzone.putTextRect(img, f"ID: {id}", (x1, y1), scale=1, thickness=1, colorR=(0,0,255))
                cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=1, colorR=(255,0,255))
            
            f.write(f"{frame},{id},{x1},{y1},{w},{h},1,-1,-1,-1\n")
        
        if show:
            cv2.imshow("Image", img)
            cv2.waitKey(1)                          
        
    f.close()

In [27]:
# Fais la détection et le tracking pour le dataset MOT20

train_datasets = os.listdir("datasets/MOT20/train")

for dataset in train_datasets:
    PATH_IMG = f"datasets/MOT20/train/{dataset}/img1"
    PATH_RESULTS = f"datasets/MOT20/train/{dataset}/results"
    detect_and_track(PATH_IMG, PATH_RESULTS)

100%|██████████| 3315/3315 [31:07<00:00,  1.78it/s]


In [5]:
#max_age=20, min_hits=1, iou_threshold=0.3, conf_yolo=0.5
max_age_list = [5, 10, 15, 25, 30, 35]
min_hits_list = [ 2, 3, 4]
iou_threshold_list = [0.1, 0.2, 0.4, 0.5]
conf_yolo_list = [0.1, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9]

for max_age in max_age_list:
    print("max_age: ", max_age)
    PATH_IMG = "datasets/MOT20/train/MOT20-01/img1"
    PATH_RESULTS = f"datasets/MOT20/train/MOT20-01/results/results_max_age"
    detect_and_track(PATH_IMG, PATH_RESULTS, max_age=max_age, file_name=f"/results_max_age_{max_age}.txt")
    
for min_hits in min_hits_list:
    print("min_hits: ", min_hits)
    PATH_IMG = "datasets/MOT20/train/MOT20-01/img1"
    PATH_RESULTS = f"datasets/MOT20/train/MOT20-01/results/results_min_hits"
    detect_and_track(PATH_IMG, PATH_RESULTS, min_hits=min_hits, file_name=f"/results_min_hits_{min_hits}.txt")
    
for iou_threshold in iou_threshold_list:
    print("iou_threshold: ", iou_threshold)
    PATH_IMG = "datasets/MOT20/train/MOT20-01/img1"
    PATH_RESULTS = f"datasets/MOT20/train/MOT20-01/results/results_iou_threshold"
    detect_and_track(PATH_IMG, PATH_RESULTS, iou_threshold=iou_threshold, file_name=f"/results_iou_threshold_{iou_threshold}.txt")
    
for conf_yolo in conf_yolo_list:
    print("conf_yolo: ", conf_yolo)
    PATH_IMG = "datasets/MOT20/train/MOT20-01/img1"
    PATH_RESULTS = f"datasets/MOT20/train/MOT20-01/results/results_conf_yolo"
    detect_and_track(PATH_IMG, PATH_RESULTS, conf_yolo=conf_yolo, file_name=f"/results_conf_yolo_{conf_yolo}.txt")


max_age:  10


100%|██████████| 429/429 [03:38<00:00,  1.96it/s]


max_age:  30


100%|██████████| 429/429 [03:39<00:00,  1.96it/s]


iou_threshold:  0.1


...

100%|██████████| 429/429 [03:38<00:00,  1.96it/s]


conf_yolo:  0.7


100%|██████████| 429/429 [03:26<00:00,  2.07it/s]


conf_yolo:  0.9


100%|██████████| 429/429 [03:25<00:00,  2.09it/s]


# Evaluation du résultat

Evaluation avec les valeurs par défaut

In [18]:
!python TrackEval/scripts/run_mot_challenge.py --NUM_PARALLEL_CORES 1 --BENCHMARK MOT20 --SPLIT_TO_EVAL train --TRACKERS_TO_EVAL YOLOSORT --METRICS HOTA --GT_FOLDER TrackEval/data/gt/mot_challenge/ --TRACKERS_FOLDER TrackEval/data/trackers/mot_challenge/


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 1                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : c:\Users\marc1\Desktop\Cours polytechnique Montréal\Hiver 2024\INF6804 - Vision par ordinateur\TP3V3\TrackEval\error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : TrackEval/data/gt/mot_challenge

# Recherche d'hyperparamètres

In [28]:
!python TrackEval/scripts/run_mot_challenge.py --NUM_PARALLEL_CORES 1 --BENCHMARK MOT20 --SPLIT_TO_EVAL train --TRACKERS_TO_EVAL YOLOSORT --METRICS HOTA --GT_FOLDER TrackEval/data/gt/mot_challenge/ --TRACKERS_FOLDER TrackEval/data/trackers/mot_challenge/


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 1                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : c:\Users\marc1\Desktop\Cours polytechnique Montréal\Hiver 2024\INF6804 - Vision par ordinateur\TP3V3\TrackEval\error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : TrackEval/data/gt/mot_challenge

In [11]:
max_age_list = [5, 10, 15, 20, 25, 30, 35]
HOTA_max_age_list = [29.523, 30.571, 31.93, 31.956, 31.832, 31.864, 32.555]
min_hits_list = [1, 2, 3, 4]
HOTA_min_hits_list = [31.569, 31.205, 30.674, 21.219]
iou_threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5]
HOTA_iou_threshold_list = [31.903, 31.972, 31.569, 31.97, 31.887]
conf_yolo_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
HOTA_conf_yolo_list =[39.508, 39.508, 38.263, 34.93, 31.569, 27.593, 17.096, 8.4722, 0]

In [14]:
#plot HOTA en fonction de max_age
plt.plot(max_age_list, HOTA_max_age_list)
plt.xlabel('max_age')
plt.ylabel('HOTA')
plt.ylim(0, 100)
plt.title('HOTA en fonction de max_age')
plt.savefig('HOTA_en_fonction_de_max_age.png')
plt.close()  

In [15]:
#plot HOTA en fonction de min_hits
plt.plot(min_hits_list, HOTA_min_hits_list)
plt.xlabel('min_hits')
plt.ylabel('HOTA')
plt.ylim(0, 100)
plt.title('HOTA en fonction de min_hits')
plt.savefig('HOTA_en_fonction_de_min_hits.png')
plt.close()

In [16]:
#plot HOTA en fonction de iou_threshold
plt.plot(iou_threshold_list, HOTA_iou_threshold_list)
plt.xlabel('iou_threshold')
plt.ylabel('HOTA')
plt.ylim(0, 100)
plt.title('HOTA en fonction de iou_threshold')
plt.savefig('HOTA_en_fonction_de_iou_threshold.png')
plt.close()


In [17]:
#plot HOTA en fonction de conf_yolo
plt.plot(conf_yolo_list, HOTA_conf_yolo_list)
plt.xlabel('conf_yolo')
plt.ylabel('HOTA')
plt.ylim(0, 100)
plt.title('HOTA en fonction de conf_yolo')
plt.savefig('HOTA_en_fonction_de_conf_yolo.png')
plt.close()

# Detection et tracking avec les meilleurs hyperparamètres

In [20]:
train_datasets = os.listdir("datasets/MOT20/train")

for dataset in train_datasets:
    PATH_IMG = f"datasets/MOT20/train/{dataset}/img1"
    PATH_RESULTS = f"datasets/MOT20/train/{dataset}/results_best"
    detect_and_track(PATH_IMG, PATH_RESULTS, conf_yolo=0.3)

100%|██████████| 3315/3315 [39:28<00:00,  1.40it/s]


In [3]:
!python TrackEval/scripts/run_mot_challenge.py --NUM_PARALLEL_CORES 1 --BENCHMARK MOT20 --SPLIT_TO_EVAL train --TRACKERS_TO_EVAL YOLOSORT --METRICS HOTA --GT_FOLDER TrackEval/data/gt/mot_challenge/ --TRACKERS_FOLDER TrackEval/data/trackers/mot_challenge/


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 1                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : c:\Users\marc1\Desktop\Cours polytechnique Montréal\Hiver 2024\INF6804 - Vision par ordinateur\TP3V3\TrackEval\error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : TrackEval/data/gt/mot_challenge

# Détection pour le dataset moodle

In [7]:
def detect_and_track_moodle(FILE_PATH_IMG, FILE_PATH_RESULTS,file_name = "/results.txt" , sort = False, show = False, className="person", max_age=20, min_hits=1, iou_threshold=0.3, conf_yolo=0.5):
    """ Différence avec le premier detect and track est le format du fichier résultat """
    
    model = YOLO("yolov8l.pt")
    classNames = model.names
    tracker = Sort(max_age=max_age, min_hits=min_hits,iou_threshold=iou_threshold)
    
    folder_files_img = os.listdir(FILE_PATH_IMG)
    if sort:
        folder_files_img = sorted(folder_files_img, key=lambda x: int(x[5:-4]))

    if not os.path.exists(FILE_PATH_RESULTS):
        os.makedirs(FILE_PATH_RESULTS)
    f = open(FILE_PATH_RESULTS+file_name, "w")

    for frame, picture in enumerate(tqdm(folder_files_img), 1):
        img = cv2.imread(f"{FILE_PATH_IMG}/{picture}")
        result = model(img, verbose=False)
        detections = np.empty((0,5))
        
        # conf_list = []
        
        for r in result:
            boxes = r.boxes
            for box in boxes:
                #Score de confiance
                conf = math.ceil(box.conf[0]*100)/100
                
                #For bounding box
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                # bbox = x1, y1, x2, y2
                # w, h = x2-x1, y2-y1
                
                #For confidence box
                cls = int(box.cls[0])
                curr_class = classNames[cls]
                
                # conf_list.append(conf)
                
                if curr_class == "cup" and conf > conf_yolo:
                    if show:
                        cvzone.putTextRect(img, f"{model.names[cls]} {conf}", (x2, y2), scale=1, thickness=1, colorR=(0,0,255))
                    currentArray = np.array([[x1, y1, x2, y2, conf]])
                    detections = np.vstack((detections, currentArray))
        
        results_tracker = tracker.update(detections)
        
        for i, res in enumerate(results_tracker):
            x1, y1, x2, y2, id = res
            x1, y1, x2, y2, id = int(x1), int(y1), int(x2), int(y2), int(id)
            w, h = x2-x1, y2-y1
            
            if show:
                cvzone.putTextRect(img, f"ID: {id}", (x1, y1), scale=1, thickness=1, colorR=(0,0,255))
                cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=1, colorR=(255,0,255))
            
            f.write(f"{frame},{id},{x1},{y1},{w},{h}\n")
        
        if show:
            cv2.imshow("Image", img)
            cv2.waitKey(1)                          
        
    f.close()

In [9]:
detect_and_track_moodle("datasets/dataset_moodle/frames", "datasets/dataset_moodle/results",sort=True , show=True, conf_yolo=0.3)

100%|██████████| 1385/1385 [11:35<00:00,  1.99it/s]


: 